In [1]:
import pandas as pd
# Use a pipeline as a high-level helper
from datasets import Dataset
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os


/Users/ethanyuxin/anaconda3/envs/ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
hf_token = os.getenv('HUGGINGFACE_API_KEY')
model_id = "meta-llama/Llama-3.2-1B-Instruct"
model_name = model_id.split("/")[-1]

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    token=hf_token
)

In [ ]:
messages = [
    {"role": "user", "content": "Who are you?"},
]
# local model in variable model
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=2048,
    temperature=0.7,
    top_p=0.9,
)
pipe(messages)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'}]}]

In [72]:
def call_pipe(pipe, section_name, content):
    messages = [{"role": "system", "content": 
                 "You are an AI assistant who will assess whether a given text is used as a background, result or method section in a scientific paper."
                 "You will be given a section name and the text, and you will need to classify the text as one of [background, result, method]."
                 "You will also need to provide a reasoning for your classification."
                 "The output should strictly be a json with the following two keys: classification, reasoning."
                #  "Example output would look like: {\"classification\": \"result\", \"reasoning\": \"This is the reasoning\"}"
    }, 
    {"role": "user", "content": f"section name: {section_name}, text: {content}"}]
    response = pipe(messages)[0]
    try:
        if response["generated_text"][2]["content"]:
            print("===" * 20)
            print(response["generated_text"][2]["content"])
            print("===" * 20)
            content = response["generated_text"][2]["content"]
            start = content.find("{")
            end = content.rfind("}") + 1
            # if model didn't end with a closing brace, add it
            if end <= 0:
                content += "\n}"
                json_str = content[start:]
            else:
                json_str = content[start:end]
            print(json_str)
            answers = json.loads(json_str)
            classification = answers["classification"]
            reasoning = answers["reasoning"]
            return classification, reasoning
        else:
            return "Invalid", "No reasoning provided"
    except Exception as e:
        print(e)
        return "Invalid", "No reasoning provided"

In [74]:
model_classification, model_reasoning = call_pipe(pipe, "Introduction", "However, how frataxin interacts with the Fe-S cluster biosynthesis components remains unclear as direct one-to-one interactions with each component were reported (IscS [12,22], IscU/Isu1 [6,11,16] or ISD11/Isd11 [14,15]).")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{
    "classification": "method",
    "reasoning": "The text is describing an experiment or observation that was conducted to study the interaction between frataxin and the Fe-S cluster biosynthesis components. It mentions specific components (IscS, IscU/Isu1, or ISD11/Isd11) and provides context about how they interact, which is typical of a method section in a scientific paper."
{
    "classification": "method",
    "reasoning": "The text is describing an experiment or observation that was conducted to study the interaction between frataxin and the Fe-S cluster biosynthesis components. It mentions specific components (IscS, IscU/Isu1, or ISD11/Isd11) and provides context about how they interact, which is typical of a method section in a scientific paper."
}


In [75]:
print(f"{model_classification}")
print(f"{model_reasoning}")

method
The text is describing an experiment or observation that was conducted to study the interaction between frataxin and the Fe-S cluster biosynthesis components. It mentions specific components (IscS, IscU/Isu1, or ISD11/Isd11) and provides context about how they interact, which is typical of a method section in a scientific paper.


In [76]:
def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    return data

In [77]:
data = preprocess("./data/train.jsonl")

In [81]:
first_partition = data[:1365]

In [82]:
labels = []
reasonings = []
raw_output = []
ids = []

for i in range(len(first_partition)):
    current_data = first_partition.iloc[i]
    ids.append(current_data.unique_id)
    classification, reasoning = call_pipe(pipe, current_data.sectionName, current_data.string)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{
  "classification": "method",
  "reasoning": "This text is describing a specific aspect of the frataxin's interaction with the Fe-S cluster biosynthesis components, which is a methodological aspect of understanding the protein's function. It is not providing new information about the protein's function or behavior, nor is it describing an experimental setup or methodology used to investigate the protein's interaction. Instead, it is reporting a specific observation or finding that has been made in the past, which is a characteristic of a method section in a scientific paper."
}
{
  "classification": "method",
  "reasoning": "This text is describing a specific aspect of the frataxin's interaction with the Fe-S cluster biosynthesis components, which is a methodological aspect of understanding the protein's function. It is not providing new information about the protein's function or behavior, nor is it describing an experimental setup or methodology used to investigate the protein's in

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{
  "classification": "result",
  "reasoning": "The text describes a study that collected data on spikes (a type of plant part) from barley, specifically focusing on the impact of certain genomic regions on root traits. The study involved sampling spikes at specific points, drying them, and storing them at a low temperature to preserve dormancy. This is an example of a scientific experiment, where the goal is to test a hypothesis or investigate a relationship between variables. The text does not describe the method or background of the study, but rather reports the outcome of the experiment. Therefore, the text is classified as a result section."
{
  "classification": "result",
  "reasoning": "The text describes a study that collected data on spikes (a type of plant part) from barley, specifically focusing on the impact of certain genomic regions on root traits. The study involved sampling spikes at specific points, drying them, and storing them at a low temperature to preserve dormanc

...

{
  "classification": "method",
  "reasoning": "This text appears to be describing a method used to investigate the relationship between hypothyroidism and Na-K-ATPase activity, specifically by citing previous studies and reporting findings that support a particular hypothesis. The text does not describe the experimental procedure, data collection, or results in detail, but rather presents a summary of the existing knowledge in the field."
}
{
  "classification": "method",
  "reasoning": "This text appears to be describing a method used to investigate the relationship between hypothyroidism and Na-K-ATPase activity, specifically by citing previous studies and reporting findings that support a particular hypothesis. The text does not describe the experimental procedure, data collection, or results in detail, but rather presents a summary of the existing knowledge in the field."
}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{
  "classification": "method",
  "reasoning": "The text describes specific methods or tools used to collect data, such as the FACIT-F scale, morning stiffness measurement, and the RA-specific Work Productivity Survey. These are not outcomes that patients report, but rather the methods used to collect data."
{
  "classification": "method",
  "reasoning": "The text describes specific methods or tools used to collect data, such as the FACIT-F scale, morning stiffness measurement, and the RA-specific Work Productivity Survey. These are not outcomes that patients report, but rather the methods used to collect data."
}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{
  "classification": "result",
  "reasoning": "The given text describes the measurement of a specific compound, Ca(21)i, with a slight modification for human use. This implies that the text is providing a factual outcome or a data point, rather than a method or a background explanation. It also suggests that the text is presenting a specific observation or measurement, which is typically what a result section in a scientific paper would contain."
{
  "classification": "result",
  "reasoning": "The given text describes the measurement of a specific compound, Ca(21)i, with a slight modification for human use. This implies that the text is providing a factual outcome or a data point, rather than a method or a background explanation. It also suggests that the text is presenting a specific observation or measurement, which is typically what a result section in a scientific paper would contain."
}
{
  "classification": "result",
  "reasoning": "The text provides empirical evidence about the

In [83]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv(f"{model_name}_first_partition.csv")